# Extract pagewise research paper contents

In [21]:
#!pip install PyMuPDF

In [22]:
#!pip install torch torchvision diffusers transformers


In [3]:
pdf_path='8.pdf'

In [4]:
#extracting text page by page without spaces

import fitz  # PyMuPDF

# Load your PDF file
#file_path = 
pdf_document = fitz.open(pdf_path)

# Initialize an empty string to store the entire text
full_text = ""

# Extract text from each page
for page_num in range(len(pdf_document)):
    page = pdf_document.load_page(page_num)  # Load the page

    # Extract text
    text = page.get_text()

    # Remove spaces before each line and join the lines together
    processed_text = ' '.join([line.strip() for line in text.split('\n')])
    
    # Append the processed text to the full_text string
    full_text += f'Page {page_num + 1} Text:\n{processed_text}\n\n'

    #print(f'Page {page_num + 1} Text:\n{processed_text}\n')

# Print the entire text
print(full_text)


Page 1 Text:
Derivative-Free Guidance in Continuous and Discrete Diffusion Models with Soft Value-Based Decoding Xiner Li 1 Yulai Zhao 2 Chenyu Wang 3 Gabriele Scalia 4 Gokcen Eraslan4 Surag Nair4 Tommaso Biancalani 4 Aviv Regev 4∗ Sergey Levine5∗ Masatoshi Uehara 4∗ 1Texas A&M University 2 Princeton University 3 MIT 4 Genentech 5 UC Berkeley Abstract Diffusion models excel at capturing the natural design spaces of images, molecules, DNA, RNA, and protein sequences. However, rather than merely -generating designs that are natural, we often aim to optimize downstream reward functions while preserving the naturalness of these design spaces. Existing methods for achieving this goal often require “differentiable” proxy models (e.g., classifier guidance or DPS) or involve computationally expensive fine-tuning of diffusion models (e.g., classifier-free guidance, RL-based fine-tuning). In our work, we propose a new method to address these challenges. Our algorithm is an iterative sampling met

# Extract pagewise Images

In [5]:
'''import fitz
import os
from PIL import Image
import io
import shutil

def clean_output_directory(output_dir):
    if os.path.exists(output_dir):
        shutil.rmtree(output_dir)
        print(f"Removed existing output directory: {output_dir}")
    os.makedirs(output_dir)
    print(f"Created new output directory: {output_dir}")

def extract_images_from_pdf(pdf_path, output_dir):
    # Clean the output directory
    clean_output_directory(output_dir)

    # Open the PDF file
    pdf_document = fitz.open(pdf_path)

    # Keep track of folders with images
    folders_with_images = set()

    # Iterate through each page
    for page_num in range(len(pdf_document)):
        page = pdf_document[page_num]
        
        # Create a folder for the current page
        page_folder = os.path.join(output_dir, f"page_{page_num + 1}")
        os.makedirs(page_folder, exist_ok=True)
        
        # Get list of images on the page
        image_list = page.get_images()
        
        # Iterate through each image on the page
        for img_index, img in enumerate(image_list):
            # Get the image data
            xref = img[0]
            base_image = pdf_document.extract_image(xref)
            image_bytes = base_image["image"]
            
            # Get the image extension
            image_ext = base_image["ext"]
            
            # Create a PIL Image object
            image = Image.open(io.BytesIO(image_bytes))
            
            # Save the image
            image_filename = f"image_{img_index + 1}.{image_ext}"
            image_path = os.path.join(page_folder, image_filename)
            image.save(image_path)
            
            print(f"Saved image: {image_path}")
            
            # Mark this folder as containing images
            folders_with_images.add(page_folder)

    # Close the PDF document
    pdf_document.close()

    # Remove empty folders
    remove_empty_folders(output_dir, folders_with_images)

def remove_empty_folders(output_dir, folders_with_images):
    for root, dirs, files in os.walk(output_dir, topdown=False):
        for dir_name in dirs:
            dir_path = os.path.join(root, dir_name)
            if dir_path not in folders_with_images and not os.listdir(dir_path):
                os.rmdir(dir_path)
                print(f"Removed empty folder: {dir_path}")

# Usage example
pdf_path = "4.pdf"
output_directory = "extracted_images"

extract_images_from_pdf(pdf_path, output_directory)'''

'import fitz\nimport os\nfrom PIL import Image\nimport io\nimport shutil\n\ndef clean_output_directory(output_dir):\n    if os.path.exists(output_dir):\n        shutil.rmtree(output_dir)\n        print(f"Removed existing output directory: {output_dir}")\n    os.makedirs(output_dir)\n    print(f"Created new output directory: {output_dir}")\n\ndef extract_images_from_pdf(pdf_path, output_dir):\n    # Clean the output directory\n    clean_output_directory(output_dir)\n\n    # Open the PDF file\n    pdf_document = fitz.open(pdf_path)\n\n    # Keep track of folders with images\n    folders_with_images = set()\n\n    # Iterate through each page\n    for page_num in range(len(pdf_document)):\n        page = pdf_document[page_num]\n        \n        # Create a folder for the current page\n        page_folder = os.path.join(output_dir, f"page_{page_num + 1}")\n        os.makedirs(page_folder, exist_ok=True)\n        \n        # Get list of images on the page\n        image_list = page.get_image

## Section wise image extraction

In [6]:
import fitz
import os
from PIL import Image
import io
import shutil
import re

def clean_output_directory(output_dir):
    if os.path.exists(output_dir):
        shutil.rmtree(output_dir)
        print(f"Removed existing output directory: {output_dir}")
    os.makedirs(output_dir)
    print(f"Created new output directory: {output_dir}")

def determine_section(text):
    text = text.lower()
    if "introduction" in text or "background" in text:
        return "Introduction"
    elif "method" in text or "methodology" in text or "experimental" in text:
        return "Methodology"
    elif "result" in text or "discussion" in text or "conclusion" in text:
        return "Results and Discussion"
    else:
        return "Other"

def extract_images_from_pdf(pdf_path, output_dir):
    clean_output_directory(output_dir)

    pdf_document = fitz.open(pdf_path)
    
    sections = ["Introduction", "Methodology", "Results and Discussion", "Other"]
    for section in sections:
        os.makedirs(os.path.join(output_dir, section), exist_ok=True)

    current_section = "Other"
    image_counter = 1

    for page_num in range(len(pdf_document)):
        page = pdf_document[page_num]
        text = page.get_text()
        
        # Check for section headers
        lines = text.split('\n')
        for line in lines:
            new_section = determine_section(line)
            if new_section != "Other":
                current_section = new_section
                break

        image_list = page.get_images()
        
        for img in image_list:
            xref = img[0]
            base_image = pdf_document.extract_image(xref)
            image_bytes = base_image["image"]
            image_ext = base_image["ext"]
            
            image = Image.open(io.BytesIO(image_bytes))
            
            image_filename = f"image_{image_counter}.{image_ext}"
            image_path = os.path.join(output_dir, current_section, image_filename)
            image.save(image_path)
            
            print(f"Saved image: {image_path}")
            image_counter += 1

    pdf_document.close()

    # Remove empty folders
    for root, dirs, files in os.walk(output_dir, topdown=False):
        for dir_name in dirs:
            dir_path = os.path.join(root, dir_name)
            if not os.listdir(dir_path):
                os.rmdir(dir_path)
                print(f"Removed empty folder: {dir_path}")

# Usage example
#pdf_path = "4.pdf"
output_directory = "extracted_images"

extract_images_from_pdf(pdf_path, output_directory)

Removed existing output directory: extracted_images
Created new output directory: extracted_images
Saved image: extracted_images\Methodology\image_1.png
Saved image: extracted_images\Methodology\image_2.png
Saved image: extracted_images\Methodology\image_3.png
Saved image: extracted_images\Methodology\image_4.png
Saved image: extracted_images\Methodology\image_5.png
Saved image: extracted_images\Methodology\image_6.png
Saved image: extracted_images\Methodology\image_7.png
Saved image: extracted_images\Methodology\image_8.png
Saved image: extracted_images\Methodology\image_9.png
Saved image: extracted_images\Methodology\image_10.png
Saved image: extracted_images\Methodology\image_11.png
Saved image: extracted_images\Methodology\image_12.png
Saved image: extracted_images\Methodology\image_13.png
Saved image: extracted_images\Results and Discussion\image_14.png
Saved image: extracted_images\Results and Discussion\image_15.png
Saved image: extracted_images\Results and Discussion\image_16.p

Here are the key changes and additions to the script:
A new function determine_section() has been added. This function tries to determine which section a given text belongs to based on keywords.
In the extract_images_from_pdf() function:
We create folders for each section at the start.
We initialize current_section as "Other" and keep track of the current section as we process the PDF.
For each page, we extract the text and check for section headers using the determine_section() function.
Images are saved in the folder corresponding to the current section.
We use a global image_counter to ensure unique image names across all sections.
The script still cleans the output directory at the start and removes empty folders at the end.
This approach has some limitations:
It relies on finding specific keywords in the text to determine sections. This may not work perfectly for all papers, especially if they use unconventional section names.
It assumes that once a section is detected, all following images belong to that section until a new section is found. This might not always be accurate.
Some images might end up in the "Other" folder if they appear before any recognizable section header.

## Trying to use LLM to summarize the individual sections

In [7]:
print(full_text)

Page 1 Text:
Derivative-Free Guidance in Continuous and Discrete Diffusion Models with Soft Value-Based Decoding Xiner Li 1 Yulai Zhao 2 Chenyu Wang 3 Gabriele Scalia 4 Gokcen Eraslan4 Surag Nair4 Tommaso Biancalani 4 Aviv Regev 4∗ Sergey Levine5∗ Masatoshi Uehara 4∗ 1Texas A&M University 2 Princeton University 3 MIT 4 Genentech 5 UC Berkeley Abstract Diffusion models excel at capturing the natural design spaces of images, molecules, DNA, RNA, and protein sequences. However, rather than merely -generating designs that are natural, we often aim to optimize downstream reward functions while preserving the naturalness of these design spaces. Existing methods for achieving this goal often require “differentiable” proxy models (e.g., classifier guidance or DPS) or involve computationally expensive fine-tuning of diffusion models (e.g., classifier-free guidance, RL-based fine-tuning). In our work, we propose a new method to address these challenges. Our algorithm is an iterative sampling met

In [45]:
import ollama

host = "127.0.0.1"
port = "11434"
client = ollama.Client(host=f"http://{host}:{port}")

# Create the prompt to categorize and summarize the sections
prompt = f"""
You are a researcher given a string containing page-wise data from a research paper:
{full_text}

#########
Summarize each section of the paper based on the provided data. The sections to be summarized are: Introduction, Methodology, and Results and Discussion. Follow these instructions for each section:

For each section found, summarize it as follows:
1. **Introduction:** Summarize the background information, research problem, objectives, and significance of the study. Highlight the research questions or hypotheses being addressed.
2. **Methodology:** Outline the research design, methods, and procedures used in the study. Include information on data collection, analysis techniques, and any specific tools or materials used.
3. **Results and Discussion:** Summarize the key findings of the research, including data and observations, and provide a summary of the interpretation of the results. Discuss how the findings relate to the research questions, hypotheses, and existing literature. Note any implications or limitations.

Do not use too many technical terms. If you are using technical terms, explain what they mean so that even non-technical people can understand the summaries.

Write the summaries using neutral language. Describe the research, methods, and findings directly, focusing solely on the content. Avoid references to sections, phrases like "the paper states" or "the section explains," and any language that implies a viewpoint or assumption. Use phrases that attribute actions and results to the research itself, such as "The research used a combination," instead of "The researchers used a combination."
"""

# Send the request to the model
res = client.chat(
    model="llama3.1",
    messages=[{"role": "user", "content": prompt}],
    options={
        "temperature": 0.5,
        "top_p": 0.9,
    }
)


In [39]:
'''import ollama

host = "127.0.0.1"
port = "11434"
client = ollama.Client(host=f"http://{host}:{port}")

# Create the prompt to categorize and summarize the sections
prompt = f"""
You are an expert in academic research tasked with summarizing a research paper. The input string contains page-wise data from the paper. Your goal is to provide clear, concise, and non-technical summaries for the following sections: Introduction, Methodology, and Results and Discussion.

Please ensure that you:

1. **Focus on Specific Sections**: Only summarize content that corresponds to the Introduction, Methodology, and Results and Discussion. Ignore any other sections, such as appendices, abstracts, or unrelated content.

2. **Infer and Summarize**: Even if the exact titles "Introduction," "Methodology," or "Results and Discussion" are not explicitly mentioned, generate summaries based on the content that most closely matches these sections:
    - **Introduction:** Summarize the background information, research problem, objectives, and significance of the study. Clearly identify the research questions or hypotheses being addressed.
    - **Methodology:** Provide a detailed outline of the research design, methods, and procedures. Include information on data collection, analysis techniques, and any tools or materials used.
    - **Results and Discussion:** Summarize the key findings, data, and observations. Provide an interpretation of the results and discuss their relevance to the research questions or hypotheses. Mention how the findings compare to existing literature and any implications or limitations.

3. **Simplify**: Use straightforward language, explaining any technical terms to ensure the summaries are understandable to non-experts.

4. **Neutral Language**: Write the summaries using neutral language. Avoid introducing opinions, assumptions, or unnecessary details. Focus solely on the content relevant to each section.

5. **Format**: Format your response exactly as follows, ensuring each section is clearly labeled:
    ```
    **Introduction:**
    [Your summarized text here]

    **Methodology:**
    [Your summarized text here]

    **Results and Discussion:**
    [Your summarized text here]
    ```

6. **Generate Content**: If the content seems to be missing, make a logical inference based on the surrounding text to construct a plausible summary for each of these sections.

**Input Data:**
{full_text}
"""

# Send the request to the model
res = client.chat(
    model="llama3.1",
    messages=[{"role": "user", "content": prompt}],
    options={
        "temperature": 0.5,
        "top_p": 0.9,
    }
)

'''

'import ollama\n\nhost = "127.0.0.1"\nport = "11434"\nclient = ollama.Client(host=f"http://{host}:{port}")\n\n# Create the prompt to categorize and summarize the sections\nprompt = f"""\nYou are an expert in academic research tasked with summarizing a research paper. The input string contains page-wise data from the paper. Your goal is to provide clear, concise, and non-technical summaries for the following sections: Introduction, Methodology, and Results and Discussion.\n\nPlease ensure that you:\n\n1. **Focus on Specific Sections**: Only summarize content that corresponds to the Introduction, Methodology, and Results and Discussion. Ignore any other sections, such as appendices, abstracts, or unrelated content.\n\n2. **Infer and Summarize**: Even if the exact titles "Introduction," "Methodology," or "Results and Discussion" are not explicitly mentioned, generate summaries based on the content that most closely matches these sections:\n    - **Introduction:** Summarize the background 

In [40]:
print(res)

{'model': 'llama3.1', 'created_at': '2024-08-28T09:36:26.8910967Z', 'message': {'role': 'assistant', 'content': 'Here are the summaries of each section:\n\n**Introduction:**\n\nThe study aims to address a research problem in the field of machine learning and data analysis. The background information suggests that current algorithms have limitations when it comes to generating samples for various domains such as images, molecules, and enhancers. The objectives of the study are to develop new methods for generating samples using histograms and to compare their performance with existing algorithms. The research questions being addressed include how well these new methods can generate high-quality samples and whether they outperform existing algorithms in terms of accuracy and efficiency.\n\n**Methodology:**\n\nThe researchers used a combination of machine learning techniques, including pre-training, best-N, deep SVDD (DSVDD), and Monte Carlo regression. They developed a novel method for g

In [41]:
'''
# Extract and display the summaries from the response
response_content = res['message']['content']

# Split the response into sections
import re


# Use regular expressions to extract sections
introduction = re.search(r'\*\*Introduction:\*\*\n\n(.*?)(?=\n\n\*\*|$)', response_content, re.DOTALL)
methodology = re.search(r'\*\*Methodology:\*\*\n\n(.*?)(?=\n\n\*\*|$)', response_content, re.DOTALL)
results_and_discussion = re.search(r'\*\*Results and Discussion:\*\*\n\n(.*?)(?=\n\n\*\*|$)', response_content, re.DOTALL)

# Store the extracted text in variables
introduction_summary = introduction.group(1).strip() if introduction else "Introduction section not available."
methodology_summary = methodology.group(1).strip() if methodology else "Methodology section not available."
results_and_discussion_summary = results_and_discussion.group(1).strip() if results_and_discussion else "Results and Discussion section not available."

# Display the summaries
print("Introduction:\n")
print(introduction_summary)
print("\nMethodology:\n")
print(methodology_summary)
print("\nResults and Discussion:\n")
print(results_and_discussion_summary)'''

'\n# Extract and display the summaries from the response\nresponse_content = res[\'message\'][\'content\']\n\n# Split the response into sections\nimport re\n\n\n# Use regular expressions to extract sections\nintroduction = re.search(r\'\\*\\*Introduction:\\*\\*\n\n(.*?)(?=\n\n\\*\\*|$)\', response_content, re.DOTALL)\nmethodology = re.search(r\'\\*\\*Methodology:\\*\\*\n\n(.*?)(?=\n\n\\*\\*|$)\', response_content, re.DOTALL)\nresults_and_discussion = re.search(r\'\\*\\*Results and Discussion:\\*\\*\n\n(.*?)(?=\n\n\\*\\*|$)\', response_content, re.DOTALL)\n\n# Store the extracted text in variables\nintroduction_summary = introduction.group(1).strip() if introduction else "Introduction section not available."\nmethodology_summary = methodology.group(1).strip() if methodology else "Methodology section not available."\nresults_and_discussion_summary = results_and_discussion.group(1).strip() if results_and_discussion else "Results and Discussion section not available."\n\n# Display the summ

In [42]:
import re

# Extract and display the summaries from the response
response_content = res['message']['content']

# Extract sections using more robust regular expressions
introduction = re.search(r'\*\*Introduction:\*\*\s*(.*?)(?=\n\n\*\*|$)', response_content, re.DOTALL)
methodology = re.search(r'\*\*Methodology:\*\*\s*(.*?)(?=\n\n\*\*|$)', response_content, re.DOTALL)
results_and_discussion = re.search(r'\*\*Results and Discussion:\*\*\s*(.*?)(?=\n\n\*\*|$)', response_content, re.DOTALL)

# Function to clean and format the extracted text
def clean_text(text):
    if text:
        cleaned = re.split(r'\n\s*Note:', text.group(1).strip(), flags=re.IGNORECASE)[0]
        return cleaned.strip()
    return "Section not available."

# Store the extracted and cleaned text in variables
introduction_summary = clean_text(introduction)
methodology_summary = clean_text(methodology)
results_and_discussion_summary = clean_text(results_and_discussion)

# Display the summaries
print("Introduction:\n")
print(introduction_summary)
print("\nMethodology:\n")
print(methodology_summary)
print("\nResults and Discussion:\n")
print(results_and_discussion_summary)


Introduction:

The study aims to address a research problem in the field of machine learning and data analysis. The background information suggests that current algorithms have limitations when it comes to generating samples for various domains such as images, molecules, and enhancers. The objectives of the study are to develop new methods for generating samples using histograms and to compare their performance with existing algorithms. The research questions being addressed include how well these new methods can generate high-quality samples and whether they outperform existing algorithms in terms of accuracy and efficiency.

Methodology:

The researchers used a combination of machine learning techniques, including pre-training, best-N, deep SVDD (DSVDD), and Monte Carlo regression. They developed a novel method for generating samples using histograms, which was then compared with the performance of existing algorithms. The data collection involved generating additional samples using 

# First version of only text on poster

In [33]:
'''import matplotlib.pyplot as plt
from PIL import Image, ImageDraw, ImageFont
import textwrap

# Create a new image with a white background
img = Image.new('RGB', (3000, 2000), color = (255, 255, 255))

# Get a drawing context
d = ImageDraw.Draw(img)

# Use a monospaced font
title_font = ImageFont.truetype('arial.ttf', 40)
body_font = ImageFont.truetype('arial.ttf', 20)

# Define the column widths
column_width = img.width // 3

# Define the title and body text
titles = ["Introduction", "Methodology", "Results and Discussion"]
summaries = [introduction_summary, methodology_summary, results_and_discussion_summary]

# Draw the titles and summaries
for i, title in enumerate(titles):
    # Draw the title
    title_width, title_height = title_font.getsize(title)
    title_x = (i * column_width) + ((column_width - title_width) // 2)
    title_y = 50
    d.text((title_x, title_y), title, font=title_font, fill=(0, 0, 0))

    # Wrap the summary text to fit within the column
    wrapped_summary = textwrap.fill(summaries[i], width=40)

    # Draw the summary
    summary_y = title_y + title_height + 50
    d.text((i * column_width + 50, summary_y), wrapped_summary, font=body_font, fill=(0, 0, 0))

# Draw the column separators
d.line([(column_width, 0), (column_width, img.height)], fill=(0, 0, 0), width=5)
d.line([(column_width * 2, 0), (column_width * 2, img.height)], fill=(0, 0, 0), width=5)

# Save the image
img.save('poster.png')'''


'import matplotlib.pyplot as plt\nfrom PIL import Image, ImageDraw, ImageFont\nimport textwrap\n\n# Create a new image with a white background\nimg = Image.new(\'RGB\', (3000, 2000), color = (255, 255, 255))\n\n# Get a drawing context\nd = ImageDraw.Draw(img)\n\n# Use a monospaced font\ntitle_font = ImageFont.truetype(\'arial.ttf\', 40)\nbody_font = ImageFont.truetype(\'arial.ttf\', 20)\n\n# Define the column widths\ncolumn_width = img.width // 3\n\n# Define the title and body text\ntitles = ["Introduction", "Methodology", "Results and Discussion"]\nsummaries = [introduction_summary, methodology_summary, results_and_discussion_summary]\n\n# Draw the titles and summaries\nfor i, title in enumerate(titles):\n    # Draw the title\n    title_width, title_height = title_font.getsize(title)\n    title_x = (i * column_width) + ((column_width - title_width) // 2)\n    title_y = 50\n    d.text((title_x, title_y), title, font=title_font, fill=(0, 0, 0))\n\n    # Wrap the summary text to fit with

# Second version : Text + Image


In [46]:
'''import matplotlib.pyplot as plt
from PIL import Image, ImageDraw, ImageFont
import textwrap
import os

def create_scientific_poster(summaries, output_directory, output_path):
    # Create a new image with a white background
    img = Image.new('RGB', (3000, 2000), color = (255, 255, 255))

    # Get a drawing context
    d = ImageDraw.Draw(img)

    # Use a monospaced font
    title_font = ImageFont.truetype('arial.ttf', 40)
    body_font = ImageFont.truetype('arial.ttf', 20)

    # Define the column widths
    column_width = img.width // 3

    # Define the title and body text
    titles = ["Introduction", "Methodology", "Results and Discussion"]

    # Draw the titles, summaries, and images
    for i, title in enumerate(titles):
        # Draw the title
        title_width, title_height = title_font.getsize(title)
        title_x = (i * column_width) + ((column_width - title_width) // 2)
        title_y = 50
        d.text((title_x, title_y), title, font=title_font, fill=(0, 0, 0))

        # Wrap the summary text to fit within the column
        wrapped_summary = textwrap.fill(summaries[i], width=40)

        # Draw the summary
        summary_y = title_y + title_height + 50
        d.multiline_text((i * column_width + 50, summary_y), wrapped_summary, font=body_font, fill=(0, 0, 0))

        # Calculate the height of the summary text
        summary_height = d.multiline_textsize(wrapped_summary, font=body_font)[1]

        # Add images from the corresponding folder
        image_folder = os.path.join(output_directory, title)
        image_y = summary_y + summary_height + 50
        if os.path.exists(image_folder):
            for image_file in os.listdir(image_folder):
                if image_file.lower().endswith(('.png', '.jpg', '.jpeg', '.gif')):
                    image_path = os.path.join(image_folder, image_file)
                    with Image.open(image_path) as section_image:
                        # Resize the image to fit the column width while maintaining aspect ratio
                        section_image.thumbnail((column_width - 100, 400))
                        image_x = i * column_width + (column_width - section_image.width) // 2
                        img.paste(section_image, (image_x, image_y))
                        image_y += section_image.height + 20  # Add some space between images

    # Draw the column separators
    d.line([(column_width, 0), (column_width, img.height)], fill=(0, 0, 0), width=5)
    d.line([(column_width * 2, 0), (column_width * 2, img.height)], fill=(0, 0, 0), width=5)

    # Save the image
    img.save(output_path)
    print(f"Poster saved as {output_path}")

# Usage example
summaries = [introduction_summary, methodology_summary, results_and_discussion_summary]
output_directory = "extracted_images"
output_path = 'scientific_poster.png'

create_scientific_poster(summaries, output_directory, output_path)'''

'import matplotlib.pyplot as plt\nfrom PIL import Image, ImageDraw, ImageFont\nimport textwrap\nimport os\n\ndef create_scientific_poster(summaries, output_directory, output_path):\n    # Create a new image with a white background\n    img = Image.new(\'RGB\', (3000, 2000), color = (255, 255, 255))\n\n    # Get a drawing context\n    d = ImageDraw.Draw(img)\n\n    # Use a monospaced font\n    title_font = ImageFont.truetype(\'arial.ttf\', 40)\n    body_font = ImageFont.truetype(\'arial.ttf\', 20)\n\n    # Define the column widths\n    column_width = img.width // 3\n\n    # Define the title and body text\n    titles = ["Introduction", "Methodology", "Results and Discussion"]\n\n    # Draw the titles, summaries, and images\n    for i, title in enumerate(titles):\n        # Draw the title\n        title_width, title_height = title_font.getsize(title)\n        title_x = (i * column_width) + ((column_width - title_width) // 2)\n        title_y = 50\n        d.text((title_x, title_y), title,

In [47]:
import matplotlib.pyplot as plt
from PIL import Image, ImageDraw, ImageFont, ImageStat
import textwrap
import os

def select_images(image_folder, num_images=3):
    if not os.path.exists(image_folder):
        return []

    image_files = [f for f in os.listdir(image_folder) if f.lower().endswith(('.png', '.jpg', '.jpeg', '.gif'))]
    
    if not image_files:
        return []

    def get_image_quality(image_path):
        with Image.open(image_path) as img:
            width, height = img.size
            stat = ImageStat.Stat(img)
            sharpness = stat.mean[:3]  # Mean RGB values as a proxy for image sharpness
            resolution = width * height
            return resolution, sharpness

    image_quality = [(img, get_image_quality(os.path.join(image_folder, img))) for img in image_files]
    sorted_images = sorted(image_quality, key=lambda x: (-x[1][0], x[1][1]))  # Sort by resolution and sharpness

    selected_images = [img for img, _ in sorted_images[:num_images]]
    return selected_images
def create_scientific_poster(summaries, output_directory, output_path):
    # Create a new image with a white background
    img = Image.new('RGB', (3000, 2000), color=(255, 255, 255))
    d = ImageDraw.Draw(img)

    # Use a monospaced font
    title_font = ImageFont.truetype('arialbd.ttf', 80)
    body_font = ImageFont.truetype('arial.ttf', 30)

    # Define layout parameters
    column_width = img.width // 3
    heading_row_height = 150
    heading_color = (100, 149, 237)  # Cornflower blue
    section_background_color = (224, 238, 255)  # Light pastel blue

    # Fill the heading row and section backgrounds
    d.rectangle([(0, 0), (img.width, heading_row_height)], fill=heading_color)
    for i in range(3):
        d.rectangle([(i * column_width, heading_row_height), ((i + 1) * column_width, img.height)], fill=section_background_color)

    titles = ["Introduction", "Methodology", "Results and Discussion"]

    for i, title in enumerate(titles):
        # Draw the title
        title_width, title_height = title_font.getsize(title)
        title_x = (i * column_width) + ((column_width - title_width) // 2)
        title_y = (heading_row_height - title_height) // 2
        d.text((title_x, title_y), title, font=title_font, fill=(255, 255, 255))

        # Wrap and draw the summary text
        wrapped_summary = textwrap.fill(summaries[i], width=30)
        summary_lines = wrapped_summary.split('\n')
        max_line_width = max([d.textsize(line, font=body_font)[0] for line in summary_lines])
        summary_x = i * column_width + (column_width - max_line_width) // 2
        summary_y = heading_row_height + 50

        for line in summary_lines:
            line_width, line_height = d.textsize(line, font=body_font)
            d.text((summary_x, summary_y), line, font=body_font, fill=(0, 0, 0))
            summary_y += line_height + 10

        # Calculate available space for images
        image_start_y = summary_y + 50
        available_height = img.height - image_start_y - 50

        # Add images
        image_folder = os.path.join(output_directory, title)
        selected_images = select_images(image_folder, num_images=3)

        if selected_images:
            # Calculate image sizes
            image_width = (column_width - 100) // 2
            image_height = available_height // 2

            for j, image_file in enumerate(selected_images):
                image_path = os.path.join(image_folder, image_file)
                with Image.open(image_path) as section_image:
                    # Resize image
                    if j < 2:
                        new_size = (image_width, image_height)
                    else:
                        new_size = (column_width - 100, image_height)
                    
                    section_image.thumbnail(new_size, Image.ANTIALIAS)

                    # Calculate position
                    if j < 2:
                        image_x = i * column_width + 50 + (j * (image_width + 10))
                        image_y = image_start_y
                    else:
                        image_x = i * column_width + 50
                        image_y = image_start_y + image_height + 10

                    # Paste the image
                    img.paste(section_image, (image_x, image_y))

    # Draw column separators
    d.line([(column_width, 0), (column_width, img.height)], fill=(0, 0, 0), width=5)
    d.line([(column_width * 2, 0), (column_width * 2, img.height)], fill=(0, 0, 0), width=5)

    # Save the image
    img.save(output_path)
    print(f"Poster saved as {output_path}")

# Usage example
summaries = [introduction_summary, methodology_summary, results_and_discussion_summary]
output_directory = "extracted_images"
output_path = 'scientific_poster.png'

create_scientific_poster(summaries, output_directory, output_path)

C:\Users\Tarun\AppData\Local\Temp\ipykernel_27484\890148101.py:52: DeprecationWarning: getsize is deprecated and will be removed in Pillow 10 (2023-07-01). Use getbbox or getlength instead.
  title_width, title_height = title_font.getsize(title)
C:\Users\Tarun\AppData\Local\Temp\ipykernel_27484\890148101.py:60: DeprecationWarning: textsize is deprecated and will be removed in Pillow 10 (2023-07-01). Use textbbox or textlength instead.
  max_line_width = max([d.textsize(line, font=body_font)[0] for line in summary_lines])
C:\Users\Tarun\AppData\Local\Temp\ipykernel_27484\890148101.py:60: DeprecationWarning: textsize is deprecated and will be removed in Pillow 10 (2023-07-01). Use textbbox or textlength instead.
  max_line_width = max([d.textsize(line, font=body_font)[0] for line in summary_lines])
C:\Users\Tarun\AppData\Local\Temp\ipykernel_27484\890148101.py:60: DeprecationWarning: textsize is deprecated and will be removed in Pillow 10 (2023-07-01). Use textbbox or textlength instead.

C:\Users\Tarun\AppData\Local\Temp\ipykernel_27484\890148101.py:91: DeprecationWarning: ANTIALIAS is deprecated and will be removed in Pillow 10 (2023-07-01). Use LANCZOS or Resampling.LANCZOS instead.
  section_image.thumbnail(new_size, Image.ANTIALIAS)


Poster saved as scientific_poster.png


## 3rd version : Text + image compression

In [35]:
'''import matplotlib.pyplot as plt
from PIL import Image, ImageDraw, ImageFont
import textwrap
import os
import random

def create_image_collage(image_folder, max_images=4, collage_size=(400, 400)):
    images = []
    for image_file in os.listdir(image_folder):
        if image_file.lower().endswith(('.png', '.jpg', '.jpeg', '.gif')):
            image_path = os.path.join(image_folder, image_file)
            images.append(Image.open(image_path))
    
    if not images:
        return None
    
    # Randomly select images if there are more than max_images
    if len(images) > max_images:
        images = random.sample(images, max_images)
    
    collage = Image.new('RGB', collage_size, (255, 255, 255))
    
    # Calculate the size for each image in the collage
    img_width = collage_size[0] // 2
    img_height = collage_size[1] // 2
    
    for i, img in enumerate(images):
        # Resize the image to fit in the collage
        img.thumbnail((img_width, img_height))
        
        # Calculate position
        x = (i % 2) * img_width
        y = (i // 2) * img_height
        
        # Paste the image into the collage
        collage.paste(img, (x, y))
    
    return collage

def create_scientific_poster(summaries, output_directory, output_path):
    # Create a new image with a white background
    img = Image.new('RGB', (3000, 2000), color = (255, 255, 255))

    # Get a drawing context
    d = ImageDraw.Draw(img)

    # Use a monospaced font
    title_font = ImageFont.truetype('arial.ttf', 40)
    body_font = ImageFont.truetype('arial.ttf', 20)

    # Define the column widths
    column_width = img.width // 3

    # Define the title and body text
    titles = ["Introduction", "Methodology", "Results and Discussion"]

    # Draw the titles, summaries, and images
    for i, title in enumerate(titles):
        # Draw the title
        title_width, title_height = title_font.getsize(title)
        title_x = (i * column_width) + ((column_width - title_width) // 2)
        title_y = 50
        d.text((title_x, title_y), title, font=title_font, fill=(0, 0, 0))

        # Wrap the summary text to fit within the column
        wrapped_summary = textwrap.fill(summaries[i], width=40)

        # Draw the summary
        summary_y = title_y + title_height + 50
        d.multiline_text((i * column_width + 50, summary_y), wrapped_summary, font=body_font, fill=(0, 0, 0))

        # Calculate the height of the summary text
        summary_height = d.multiline_textsize(wrapped_summary, font=body_font)[1]

        # Create and add image collage
        image_folder = os.path.join(output_directory, title)
        if os.path.exists(image_folder):
            collage = create_image_collage(image_folder)
            if collage:
                collage_y = summary_y + summary_height + 50
                collage_x = i * column_width + (column_width - collage.width) // 2
                img.paste(collage, (collage_x, collage_y))

    # Draw the column separators
    d.line([(column_width, 0), (column_width, img.height)], fill=(0, 0, 0), width=5)
    d.line([(column_width * 2, 0), (column_width * 2, img.height)], fill=(0, 0, 0), width=5)

    # Save the image
    img.save(output_path)
    print(f"Poster saved as {output_path}")

# Usage example
summaries = [introduction_summary, methodology_summary, results_and_discussion_summary]
output_directory = "extracted_images"
output_path = 'scientific_poster.png'

create_scientific_poster(summaries, output_directory, output_path)'''

'import matplotlib.pyplot as plt\nfrom PIL import Image, ImageDraw, ImageFont\nimport textwrap\nimport os\nimport random\n\ndef create_image_collage(image_folder, max_images=4, collage_size=(400, 400)):\n    images = []\n    for image_file in os.listdir(image_folder):\n        if image_file.lower().endswith((\'.png\', \'.jpg\', \'.jpeg\', \'.gif\')):\n            image_path = os.path.join(image_folder, image_file)\n            images.append(Image.open(image_path))\n    \n    if not images:\n        return None\n    \n    # Randomly select images if there are more than max_images\n    if len(images) > max_images:\n        images = random.sample(images, max_images)\n    \n    collage = Image.new(\'RGB\', collage_size, (255, 255, 255))\n    \n    # Calculate the size for each image in the collage\n    img_width = collage_size[0] // 2\n    img_height = collage_size[1] // 2\n    \n    for i, img in enumerate(images):\n        # Resize the image to fit in the collage\n        img.thumbnail((

# Generating the poster using stable diffusion

In [36]:
'''import torch
from diffusers import StableDiffusionPipeline
from PIL import Image
import requests
from io import BytesIO
import os

def load_local_images(directory):
    images = []
    for root, _, files in os.walk(directory):
        for file in files:
            if file.lower().endswith(('png', 'jpg', 'jpeg')):
                image_path = os.path.join(root, file)
                images.append(Image.open(image_path).convert("RGB"))
    return images

# Set the path to your local image directories
image_directory = "extracted_images"

# Load all images from the specified directory
images = load_local_images(image_directory)
'''

In [37]:
#images

[<PIL.Image.Image image mode=RGB size=227x253>,
 <PIL.Image.Image image mode=RGB size=500x209>,
 <PIL.Image.Image image mode=RGB size=500x207>,
 <PIL.Image.Image image mode=RGB size=500x395>,
 <PIL.Image.Image image mode=RGB size=500x256>,
 <PIL.Image.Image image mode=RGB size=500x398>,
 <PIL.Image.Image image mode=RGB size=500x398>,
 <PIL.Image.Image image mode=RGB size=500x298>,
 <PIL.Image.Image image mode=RGB size=500x298>,
 <PIL.Image.Image image mode=RGB size=500x298>,
 <PIL.Image.Image image mode=RGB size=500x298>,
 <PIL.Image.Image image mode=RGB size=500x396>,
 <PIL.Image.Image image mode=RGB size=500x254>,
 <PIL.Image.Image image mode=RGB size=500x262>,
 <PIL.Image.Image image mode=RGB size=500x262>,
 <PIL.Image.Image image mode=RGB size=500x262>,
 <PIL.Image.Image image mode=RGB size=512x512>,
 <PIL.Image.Image image mode=RGB size=512x512>,
 <PIL.Image.Image image mode=RGB size=512x512>,
 <PIL.Image.Image image mode=RGB size=512x512>,
 <PIL.Image.Image image mode=RGB size=51

In [38]:
'''from PIL import Image, ImageDraw

def add_border(image, border_size=10, border_color=(0, 0, 0)):
    # Create a new image with border
    bordered_image = Image.new('RGB', 
                               (image.width + 2 * border_size, image.height + 2 * border_size), 
                               border_color)
    bordered_image.paste(image, (border_size, border_size))
    return bordered_image
def create_composite_image_with_borders(images, border_size=10, border_color=(0, 0, 0)):
    if not images:
        raise ValueError("No images to combine.")
    
    # Add borders to each image
    bordered_images = [add_border(img, border_size, border_color) for img in images]

    # Calculate width and height for the final poster
    widths, heights = zip(*(i.size for i in bordered_images))
    
    # Create a new image with the maximum width and the sum of heights
    total_width = max(widths)
    total_height = sum(heights)
    
    new_image = Image.new('RGB', (total_width, total_height), (255, 255, 255))
    
    y_offset = 0
    for img in bordered_images:
        new_image.paste(img, (0, y_offset))
        y_offset += img.height

    return new_image
# Create the composite image with borders
composite_image = create_composite_image_with_borders(images, border_size=10, border_color=(0, 0, 0))
composite_image.save("composite_image_with_borders.png")'''

In [39]:
'''prompt = f"""
Create a high-quality scientific poster layout with three distinct vertical columns, each dedicated to a specific section: 'Introduction,' 'Methodology,' and 'Results and Discussion.' The poster should be visually appealing and suitable for a professional conference setting, with clean lines, clear headings, and well-organized content.

- **Introduction**: In the first column, summarize the key points of the research as described below:
  
  _'{introduction_summary}'_
  
  Ensure the text is concise and informative, providing context and background for the study.

- **Methodology**: The second column should focus on the research methods, summarizing the essential steps and approaches as described below:
  
  _'{methodology_summary}'_
  
  Include any necessary diagrams or flowcharts that clarify the experimental setup or procedures.

- **Results and Discussion**: In the third column, present the key findings and analysis as detailed below:
  
  _'{results_and_discussion_summary}'_
  
  Highlight any significant trends, correlations, or conclusions drawn from the data.

In each column, ensure the provided images from the image given is prominently placed exactly where they are relevant, without any alterations. The images should be seamlessly integrated into the design without any modifications (They should, aligned with the text, and maintaining their original clarity and quality. Use appropriate captions under each image to describe its relevance to the section's content.

The overall design should be cohesive, with a consistent color scheme and font choice that enhances readability while maintaining a professional and academic tone.
"""

'''

In [45]:
#!pip install accelerate

     ---------------------------------------- 0.0/315.1 kB ? eta -:--:--
     -------------------------------------  307.2/315.1 kB 9.6 MB/s eta 0:00:01
     -------------------------------------- 315.1/315.1 kB 4.8 MB/s eta 0:00:00


In [46]:
'''# Load the Stable Diffusion model
device = "cuda" if torch.cuda.is_available() else "cpu"
pipe = StableDiffusionPipeline.from_pretrained("CompVis/stable-diffusion-v1-4")
pipe = pipe.to(device)
# Generate the final poster
composite_image_path = "composite_image_with_borders.png"
composite_image = Image.open(composite_image_path).convert("RGB")

# Run Stable Diffusion
result = pipe(prompt=prompt, init_image=composite_image, strength=0.75, guidance_scale=9,height=720, width=480).images[0]
result.save("final_poster.png")
print("Saved final poster image: final_poster.png")'''

Cannot initialize model with low cpu memory usage because `accelerate` was not found in the environment. Defaulting to `low_cpu_mem_usage=False`. It is strongly recommended to install `accelerate` for faster and less memory-intense model loading. You can do so with: 
```
pip install accelerate
```
.


Loading pipeline components...:   0%|          | 0/7 [00:00<?, ?it/s]

Token indices sequence length is longer than the specified maximum sequence length for this model (558 > 77). Running this sequence through the model will result in indexing errors
The following part of your input was truncated because CLIP can only handle sequences up to 77 tokens: ["summarize the key points of the research as described below : _' the introduction provides background information on the importance of generating high - quality samples in various domains such as images, molecules, and enhancers. it highlights the challenges of current methods and the need for a more efficient approach. the research aims to address this problem by employing a novel method that uses convgru as the value function model. the objectives are to evaluate the performance of generated samples using histograms and compare it with existing algorithms.'_ ensure the text is concise and informative, providing context and background for the study. - ** methodology **: the second column should focus on 

  0%|          | 0/50 [00:00<?, ?it/s]

Saved final poster image: final_poster.png


# Making bland poster beautiful using stable diffusion

In [1]:
'''import torch
from diffusers import StableDiffusionPipeline
from PIL import Image
import requests
from io import BytesIO
import os

# Load the Stable Diffusion model
device = "cuda" if torch.cuda.is_available() else "cpu"
pipe = StableDiffusionPipeline.from_pretrained("CompVis/stable-diffusion-v1-4")
pipe = pipe.to(device)
# Generate the final poster
composite_image_path = "scientific_poster.png"
composite_image = Image.open(composite_image_path).convert("RGB")


prompt = f"""
Enhance the design of this scientific poster while retaining the existing text in all three sections: Introduction, Methodology, and Results & Discussion. Ensure that the text remains unchanged and perfectly readable. Improve the overall aesthetics without altering the images or content in the three columns. Focus on creating a more visually appealing and professional design that highlights the existing structure and information effectively
"""

# Run Stable Diffusion
result = pipe(prompt=prompt, init_image=composite_image, strength=0.75, guidance_scale=9,height=720, width=480).images[0]
result.save("final_poster.png")
print("Saved final poster image: final_poster.png")'''

Loading pipeline components...:   0%|          | 0/7 [00:00<?, ?it/s]

C:\Users\Tarun\Anaconda3\envs\Whisper\lib\site-packages\transformers\tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


  0%|          | 0/50 [00:00<?, ?it/s]

Saved final poster image: final_poster.png


# Generating a poster backdrop with stable diffusion based on the research paper


In [2]:
'''import torch
from diffusers import StableDiffusionPipeline

# Load the Stable Diffusion model
device = "cuda" if torch.cuda.is_available() else "cpu"
pipe = StableDiffusionPipeline.from_pretrained("CompVis/stable-diffusion-v1-4")
pipe = pipe.to(device)

field = 'Computer Science'
# Define the prompt
prompt = """
Design a formal and professional {field} scientific poster backdrop with a clean, elegant outer border. The backdrop should be minimalistic, using a sophisticated color palette. There should be no sections, only a refined outer border, suitable for adding text and images later.
"""

# Run Stable Diffusion
result = pipe(prompt=prompt, guidance_scale=9, height=720, width=480).images[0]
result.save("final_poster.png")
print("Saved final poster image: final_poster.png")
'''

Loading pipeline components...:   0%|          | 0/7 [00:00<?, ?it/s]

  0%|          | 0/50 [00:00<?, ?it/s]

Saved final poster image: final_poster.png
